# Introduction

This notebook allows you to run a simplified version of the ANTARESS workflow, for the following purpose:
- clean and correct 2D spectral time-series recorded by the detector
- convert the cleaned and corrected 2D spectral time seires to 1D spectra and a stellar master spectrum

This notebook is used to process time-series of 2-dimensional echelle spectra. This notebook allows for a complete correction of the observational spectra and to produce a 1D master spectrum and the individual 1D spectra for each exposure. To try the workflow, place your observational dataset in the directory of your choice (named, for example, `Working_dir/`), and set the `working_path` to the directory containing your data.

Some options and functionallities are not included in this notebook to keep it slim and easy to use. For targets with multiple visists it is espesecially recommended to use the [configuration files](https://obswww.unige.ch/~bourriev/antaress/doc/html/installation.html).

In [1]:
import ANTARESS_nbook_bground
from antaress.ANTARESS_launch.ANTARESS_launcher import ANTARESS_launcher

input_nbook = ANTARESS_nbook_bground.init('SP_reduc')
input_nbook['working_path'] ='/Users/erikfriden/Desktop/antaress/working_dir/'

2024-10-29 14:58:59,547 - WARNING - File .dacerc not found. You are requesting data in public mode. To change this behaviour, create a .dacerc file in your home directory and fill it with your API key. More infos on https://dace.unige.ch


# System parameters

**Parameters of the host star**

Run this cell to define the host star. 
- `star_name` (string): Name of the star. Should not contain spaces/dashes.
- `Rs` (float): Radius of the star, in $R_\odot$.
- `vsini` (float): Projected stellar surface velocity, in km/s. The value given here is only used as guess for the fits, as it will eventually be derived from your analysis.
- `ld_u1` (float): First coefficient of the quadratic limb-darkening.
- `ld_u2` (float): Second coefficient of the quadratic limb-darkening.

In [2]:
input_nbook['par'].update({
    'star_name' : 'TOI421',
    'Rs'        : 0.866,
    'vsini'     : 1.8,
    'sysvel'    : 80.,
    'ld_u1'     : 0.48,
    'ld_u2'     : 0.18
})
ANTARESS_nbook_bground.init_star(input_nbook)

**Parameters of transiting planet**

Run this cell to set the transiting planet on which the RM analysis is applied. 
- `planet_name` (string): Name of the transiting planet. Should not contain spaces/dashes.
- `period` (float): Orbital period, in days.
- `T0` (float): Mid-transit time, in ${\rm BJD}_{\rm TBD}$.
- `ecc` (float): Orbital eccentricity.
- `long_per` (float): Longitude of periastron, in degrees. If the orbit is circular, fill in `90.`.
- `Kstar` (float): RV semi-amplitude, in m/s.
- `aRs` (float): Scaled separation $a/R_\star$.
- `incl` (float): Orbital inclination, in degrees.
- `RpRs` (float): Planet-to-star radius ratio $R_{\rm p}/R_\star$.

In [3]:
input_nbook['par'].update({
    'planet_name' : 'TOI421c',
    'period'      : 16.067541,
    'T0'          : 2459195.30741,
    'ecc'         : 0.19,
    'long_per'    : 102.,
    'Kstar'       : 4.1,
    'aRs'         : 29.0546178,
    'incl'        : 88.30373,
    'RpRs'        : 0.0540
})
ANTARESS_nbook_bground.init_pl(input_nbook,'main')

**Parameters of additional planets in the system**

Run this cell to set additional planets, which will only contribute to the computation of the star Keplerian motion. 

The names and definitions of the parameters are the same as for the main planet (`aRs`, `incl`, `RpRs` are not required).

Copy the cell to add further planets

In [4]:
input_nbook['par'].update({
    'planet_name' : 'TOI421b',
    'period'      : 5.197576,
    'T0'          : 2459189.7341,
    'ecc'         : 0.13,
    'long_per'    : 140.,
    'Kstar'       : 2.83
})
ANTARESS_nbook_bground.init_pl(input_nbook,'sec')

# Dataset properties

**Visit**

Run this cell to define the visit to be processed. It must overlap with the transit of the `transiting planet`.
- `instrument` (string): Instrument of the visit (e.g. `ESPRESSO`, `NIRPS_HE`...).
- `night` (string): Night of the visit. It is preferred to use the standard format, i.e. `YYYYMMDD`.
- `data_dir` (string): Path relative to `wokring_dir` to the directory holding the observed .fits files (here, set to `observations/S2D` to retrieve the 2D spectra with ANTARESS)

In [5]:
input_nbook['par'].update({
    'instrument'  : 'ESPRESSO',
    'night'       : '20231106',
    'data_dir'    : 'observations/S2D',
})
ANTARESS_nbook_bground.add_vis(input_nbook)

# Processing of spectral data

<a id='inst_cal'></a>
**Instrumental calibration**

The input spectra in `'data_dir'` are assumed to be corrected for standard nstrumental effects such as flat field, blaze, and background.

- `blaze`: `True` to measure calibration profile and detector noise from blazed data, S2D_BLAZE fits file must be provided in the same directory as S2D data.
- `calc_gcal`: `True` to calculate results and `False` to retrieve previous results.

Run this cell to measure the instrumental calibration. It is used in some modules for the rescaling of photoelectrons to their original photoelectron levels. 

In [6]:
input_nbook['sp_reduc'].update({'blaze'    : True,
                               'calc_gcal' : False})
ANTARESS_nbook_bground.inst_cal(input_nbook, plot=False)

**Launching the ANTARESS workflow**

Run this cell to launch the ANTARESS workflow. During the spectral reduction it is a good practice to inspect the corrections after each step since the next correction will be applied to spectra after the previous correction step. 

In [7]:
input_nbook['sp_reduc'].update({'proc_data' : True})
ANTARESS_nbook_bground.processing_mode(input_nbook)

In [8]:
#ANTARESS_launcher(working_path=input_nbook['working_path'], nbook_dic = input_nbook, exec_comm=False)

<a id='process_data'></a>
**ANTARESS in calculation/retrieval mode**

To avoid unecessary computation time we deactivate the reading in of the observed data after this step since all corrections will be appied to the corrected spectra from the previous step. 

However, if pixels are later being masked and excluded from the correction after this step you will have to rerun the previous step in calculation mode again. At this point, the cell for masking of pixels would already have been executed, and you only have to activate and run this cell again. After you can rerun the instrumental calibration by running that cell above and launching the ANTARESS workflow. The instrumental calibration will then be performed on the observational data exluding the masked pixels. Again, deactivate the processing of observaitonal data in this cell and continue to run the cells below and perform the corrections once more but now with the pixels masked. Inspect the new corrections and continue with the workflow.

In [9]:
input_nbook['sp_reduc'].update({'proc_data' : False})
ANTARESS_nbook_bground.processing_mode(input_nbook)

**Telluric correction**

Run this cell to perform the telluric correction. 

- `tell_species` (list, string): choose which molecules to be used in correction (H$_2$O, O$_2$, CH$_4$, CO, CO$_2$)       
- `tell_thresh` (float): threshold for contrast of telluric lines to be masked as nan



In [10]:
input_nbook['sp_reduc'].update({
    'tell_species'     : ['H2O', 'O2'],
    'tell_thresh'      : 0.1
})

# Calculate results (True) or retrieve previous results (False)
input_nbook['sp_reduc'].update({'calc_tell' : False})

ANTARESS_nbook_bground.tell_corr(input_nbook, plot=False)

**Adding a spectral plotting option here**

Run this cell to plot either the transmission spectrum `sp_trans` or the raw spectrum `sp_raw` before and after the telluric correction.




In [11]:
input_nbook['sp_reduc'].update({
    'sp_raw'     : False,
    'trans_sp'   : False,
    'wav_range'  : [3000,7000],
    'y_range'    : None,
    'norm_prof'  : True,
    'plot_master': False,
    'multi_exp'  :True
})


In [12]:
#ANTARESS_launcher(working_path=input_nbook['working_path'], nbook_dic = input_nbook, exec_comm=False)

**Masking of pixels**

Run this cell to mask pixels/ranges to exclude from the correction. 

Indicate which spectral orders to be masked and the corresponding wavelength range to mask

- `order` (list): spectral orders to be masked, e.g., [10,24]
- `range` (list): wavelength range to be masked for the spectral orders in the list `order`. The list must have the same length as the list in `order`, the format as [[low 10, high 10], [low 24, high 24]], for the range to be masked in spectral order in `order` as [10, 24].

If you chosed to exclude spectral ranges from the correction, run this cell to indicate spectral orders or ranges to mask and then go to [ANTARESS retrieval mode](#process_data), activate `proc_data` and launch ANTARESS from the [Instrumental calibration](#inst_cal).


In [13]:
input_nbook['sp_reduc'].update({
    'order' : [],
    'range'  : [[]]
})

ANTARESS_nbook_bground.mask_pix(input_nbook)

**Global flux balance correction**

A first correction based on the flux ratio between each exposure and its visit master computed as the median of all exposures.

- `sigma_clip` (bool): activate to remove outliers (recommended to manually exclude orders in `ord_excl_fit`)
- `nord` (float): indicate number of spectral orders, if not known set it to `None` run this cell and the cell below to plot the flux colour balance. The top axis indicates the number of spectral orders.
- `ord_excl_fit` (list): indicate which orders to be excluded from fit (not used if sigma clipping is activated)
- `phantom_bins` (float): the range of phantom bins in $\nu$ to avoid divergence in the blue part of the spectrum.
- `unc_scaling` (float): variance of fitted bins is scaled to the chosen power (0 = equal weights: 1 = no scaling (original weights), increase to give more weight to data with low errors.
- `fit_mode` (string): `'pol'` for polynomial function or `'spline'` for 1D-smoothing spline
- `pol_deg` (integer): polynomial function degree (not used in when 'spline' is chosen)
- `smooth_fac` (float): spline smoothing factor, increase to smooth

Phantom bins can be used to mirror a linear fitting onto the blue part of the spectrum. This can be used to avoid divergence at the end of the spectrum. To find the best parameters for the model an iterative approach is best used varying the different parameters.


In [14]:
input_nbook['sp_reduc'].update({
    'sigma_clip'    : False,
    'nord'          : 170,
    'ord_excl_fit'  : [0,1,88,89,90,91,145,146,147,164,165],
    'phantom_range' : 10.,##
    'unc_scaling'   : 0.25,
    'fit_mode'      : 'spline',
#When using the polynomial function
    'pol_deg'       : 4.,
#When using the 1-D smoothing spline
    'smooth_fac'    : 1.5e-5
})

#True to calculate the correction, False to retrieve last result
input_nbook['sp_reduc'].update({
    'calc_Fbal' : False
})

ANTARESS_nbook_bground.fbal_corr(input_nbook)

**Plotting the global flux colour balance**

Plot the flux balance correction while separating the exposures (True), or plot all exposures on top of each other (Flase)

In [15]:
input_nbook['plots'].update({
    'gap_exp': False
})

#ANTARESS_nbook_bground.plot_fbal_corr(input_nbook)

In [16]:
#ANTARESS_launcher(working_path=input_nbook['working_path'], nbook_dic = input_nbook, exec_comm=False)

**Cosmics correction**

Run this cell to perform the cosmic hits correction.

The cosmic rays are detected by comparing the relative flux between exposures taken before and after. The threshold sets a limit for which a cosmic ray is considered detected. If the relative flux difference is higher than the cosmic threshold multiplied by the standard deviation of the measured or comparison spectra, a cosmic ray is considered detected.

- `align_method` (string): choose alignment method, `kep` to correct for the Keplerian curve with information entered in the system properties, `pip` to use pipeline RVs if available. Keplerian curve is preferred to avoid biased RVs due to RM effect.     
- `ncomp` (integer): number of comparison spectra to be used for the cocmic detection     
- `thresh` (float): set the threshold for which a cosmic hit is considered detected


In [17]:
input_nbook['sp_reduc'].update({
    'calc_cosm'    : False,
    'align_method' : 'kep',
    'ncomp'        : 10,
    'thresh'       : 10.
})

ANTARESS_nbook_bground.cosm_corr(input_nbook, plot=False)

In [18]:
#ANTARESS_launcher(working_path=input_nbook['working_path'], nbook_dic = input_nbook, exec_comm=False)

**ESPRESSO "wiggles" correction**

Run this cell to characterise and correct wiggles; only the screening and filter options are available in this notebook. To use the analytical version use the [configuration files](https://obswww.unige.ch/~bourriev/antaress/doc/html/installation.html).

In this cell, you use screening to visualise the wiggles and, from there, choose what spectral ranges in $\nu$ to include in the fitting. Generally, you will see large spurious features at the centre of the transmission spectrum located at the end and start frequencies of the red and blue detectors. Additionally, at the blue end of the spectrum, the noise levels will be much larger than the amplitude of the wiggle pattern. Hence, parts of these regions generally need to be removed from the fitting. The filter method is then used to characterise the wiggles using a Savitzky-Golay filter of the binned transmission spectrum in each exposure.

Screening:
- `screening` (bool): activate/deactivate the screening and plotting
- `fit_range` (list): indicate lower and upper boundaries of ranges to include in fit, the format is [[low 1, high 1], [low 2, high 2], [etc.]] in $\nu$, leave empty to use the full range
- `y_range` (list): list including the lower and upper y limit, if None automatic scaling is applied

Filter:
- `filter` (bool): activate/deactivate the filter correction and characterisation
- `window` (float): size of smoothing window in $\nu$
- `deg` (integer): polynomial degree used to fit the smoothed spectrum

When using the filter correction, be careful to choose an appropriate combination of window size and polynomial degree to avoid fitting noise and spurious features in the data.

In [19]:
#Screening
input_nbook['sp_reduc'].update({
    'screening'    : False,
    'fit_range'    : [[]],
    'y_range'      : None
})

#Filter
input_nbook['sp_reduc'].update({
    'filter'       : False,
    'window'       : 0.2,
    'deg'          : 3,
})

#To calculate wiggles set to True, to retrieve previous result set to False. First time it has to be True. 
#Set corr_wig to False to deactivate the module
input_nbook['sp_reduc'].update({
    'corr_wig':False,
    'calc_wig':True
})

ANTARESS_nbook_bground.wiggle_corr(input_nbook)

In [20]:
#ANTARESS_launcher(working_path=input_nbook['working_path'], nbook_dic = input_nbook, exec_comm=False)

# Spectral processing

At this step, the original S2D spectra has been cleaned from instrumental effects, telluric contamination, cosmic ray hits, and consits of a time-series of cleaned 2D-spectra. The following steps will allow us co compute a 1D master stellar spectrum from out-of-transit data, that could be used either directly for analysis or to build/or compute a custom CCF mask.

The following steps includes procedures for aligning, scaling, and weighting the spectra to finally produce a master 1D stellar spectrum.

**Detrending of spectral lines**

If this is the first time processing the spectral time series, skip this step since the trend characteristion will be performed in a separate notebook and is performed on the CCFs computed here. After identifying any trends in the [characterisation notebook](link) you can activate the module below and define the corrections to be applied on the S2D spectra, and then recompute the CCFs from the detrended S2D specta.

To perform the detrending in this step, the [characterisation notebook](link) must have been used to determine any trends in the data and appropriate corrections first. Here you will use the values for the coefficients determined using the [characterisation notebook](link). Define which proeprty to correct and the corresponding dependence. If multiple properties are being corrected add them to the list.

- `use` (bool): set to True to perform a trend correction
- `prop` (list, str): list each property to correct with the correcsponding dependece. Properties to correct here is either RV or contrast (ctrst), and as a function of either phase or snr (with ESPRESSO snrQ)
- `coeff` (list,list): indicate coefficients used for each model, add coefficients with decreasing order, add a new list if more than one property is corrected. The RV coefficient should be in km/s.


In [21]:
input_nbook['par'].update({
    'use'  : False,
    'prop' : ['ctrst_snrQ', 'RV_phase'],
    'mode' : ['pol', 'pol'],
    'coeff': [[-1.109190e-05], [-6.840494e-02]]
})

ANTARESS_nbook_bground.detrend(input_nbook)

**Compute CCF from spectra**

Comupute CCF from input spectra using used defined CCF masks.

 + `start_RV` (float): start RV for CCF computation in km/s, in stellar restframe
 + `end_RV` (float): end RV for CCF computation in km/s, in stellar restframe
 + `dRV` (float): step size in RV, use `None` for instrumental resolution. Using smaller steps than resolution will introduce correlations.
 + `mask_path` (string): location where the mask is stored + filename (relative to `'working_path'`)


In [24]:
input_nbook['sp_reduc'].update({
    'start_RV' : -150.,
    'end_RV'   : 150.,
    'dRV'      : None,
    'mask_path': 'CCF_masks/ESPRESSO_new_G9.fits',
    'calc_CCF' : True
})

ANTARESS_nbook_bground.DI_CCF(input_nbook)

**Systemic velocity**

To align the system in the star rest frame you can first use a systemic velocity for the star from the literature. However, for the final analysis it has to be measured from the same dataset and for each epoch. This will be done in a separate notebook.

Set `gamma` to the systemic velocity and proceed with the analysis. (Use measured velocity if available otherwise use literature values as a first estimate)

In [25]:
input_nbook['par']['gamma'] = 79.403747
ANTARESS_nbook_bground.set_sysvel(input_nbook)

**Alignment in star rest frame**

Run this cell to align the disk-integrated S2D spectra in the star rest frame.

S2D spectra are originally defined in the Sun barycentric rest frame. This module shifts them by the systemic rv you defined above, and by the Keplerian rv motion induced by the planets you set up for this system.

This step is needed when aligning the system for building the master spectrum from out-of-transit exposures.

In [26]:
ANTARESS_nbook_bground.align_prof(input_nbook)

**Flux scaling**

Run this cell to scale the disk-integrated Spectra to their correct relative flux level over the transit. 

The scaling depends on the limb-darkening coefficients and planet-to-star radius ratio you entered in the `System parameters`.

This step is needed when calculating the master stellar spectrum for weighting when building the master spectrum.

In [27]:
ANTARESS_nbook_bground.flux_sc(input_nbook)
ANTARESS_nbook_bground.DImast_weight(input_nbook)

In [28]:
ANTARESS_launcher(working_path=input_nbook['working_path'], nbook_dic = input_nbook, exec_comm=False)

****************************************
Launching ANTARESS
****************************************

Multi-threading: 12 threads available
Running with observational data
Study of: TOI421c
Accounting for Keplerian motion from all planets
Automatic definition of T14[TOI421c]=2.76 h
Default nsub_Dpl[TOI421c]=26

-----------------------
Processing instrument : ESPRESSO
-----------------------
  Reading and initializing 2D echelle spectra
   > Errors propagated from raw data
   > Data processed on individual spectral tables for each exposure
         Retrieving data
         Processing visit 20231106
           Exposures do not share a common spectral table
           54 exposures
   > Estimating instrumental calibration
         Retrieving data for 20231106
   > Correcting spectra for tellurics
         Retrieving data for 20231106
   > Calculating stellar masters
         Calculating data
   > Correcting spectra for global flux balance
         Calculating data
   > Correcting spectra f

TypeError: 'NoneType' object is not subscriptable

In [ ]:
%tb

**Below this cell we remove the lines and use in a separate notebook.**

In [ ]:
ANTARESS_nbook_bground.calc_DImast(input_nbook) # For weightinh when stacking out-of-transit exposures

**2D to 1D conversion of disk-integrated spectra**

Here we compute the 1 dimensional spectra for each exposure in the time series.
This process is computationally heavy and requires high memory usage, so if it is a problem you can cahnge the number of cores to use with `ncores`.

- `ncores` (integer): number of cores to use
- `wav_step` (float): wavelength are unifomrly spaced in $\ln(\lambda)$ where $\text{d}[\ln(\lambda)] = \text{d} \lambda/\lambda$.
- `wav_start` (float): start wavelength of spectral table
- `wav_end` (float): end wavelength of spectral table

In [ ]:
input_nbook['sp_reduc'].update({
    'ncores'    : None,
    'wav_step'  : 0.01/6000.,
    'wav_start' : 3000.,
    'wav_end'   : 9000.,
})

ANTARESS_nbook_bground.convert_to_1D(input_nbook, plot=False)

**Binning disk-integrated 1D spectra**

Run this cell to bin all out of transit exposures into one 1D-master stellar spectrum.


In [ ]:
ANTARESS_nbook_bground.build_1D_master(input_nbook, plot=False)